reference https://www.kaggle.com/gaoyuan19930220/xgboost

In [3]:
import keras 
import numpy as np
import pandas as pd
import os
import sklearn
import xgboost as xgb
import operator
import optuna

from functools import partial
from matplotlib import pylab as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, StratifiedKFold


import warnings
warnings.filterwarnings('ignore')
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [46]:
def preproceed(dataframe):
    # 欠損値の処理
    dataframe.replace({'Nan':pd.np.nan,'nan':pd.np.nan},inplace=True)
#     dataframe.fillna(dataframe.mean(numeric_only=True),inplace=True)
#     dataframe['Fare'].fillna(dataframe['Fare'].mean(),inplace=True)
#     dataframe['Embarked'].fillna(dataframe['Embarked'].mode().loc[0],inplace=True)
    
    #自分以外の家族の人数及び有無を追加
    dataframe['NumFamily'] = dataframe['SibSp'] + dataframe['Parch']
    dataframe['IsFamily'] = dataframe['NumFamily'] != 0

    #質的変数をダミー変数に
    dataframe['IsPclass1'] = dataframe['Pclass'] == 1
    dataframe['IsPclass2'] = dataframe['Pclass'] == 2
    dataframe['IsPclass2'] = dataframe['Pclass'] == 3
    dataframe['IsEmbarkedS'] = dataframe['Embarked'] == 'S'
    dataframe['IsEmbarkedC'] = dataframe['Embarked'] == 'C'
    dataframe['IsEmbarkedQ'] = dataframe['Embarked'] == 'Q'
    
    
    #質的変数を変換'
    dataframe['Sex'] = dataframe['Sex'].apply(lambda x: 1 if x == 'male' else 0)
#     dataframe['Embarked'] = dataframe['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2}).astype('int')
    dataframe[np.asarray(dataframe.select_dtypes('bool').columns)] = dataframe[np.asarray(dataframe.select_dtypes('bool').columns)].astype('int')
    
    #いらない列(Cabin,Name,PassengerId,Ticket)の削除
    dataframe.drop(['Cabin','Name','Ticket','Embarked','Pclass'],axis=1,inplace=True)

In [47]:
#csvデータの呼び出し場所
loadpath = "../input/"
#csvデータの保存場所
savepath = "../output/"

In [48]:
df = pd.read_csv(loadpath + "train.csv")
df_test = pd.read_csv(loadpath + "test.csv")

df.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)

train_index = df.index
test_index = df_test.index

preproceed(df)
preproceed(df_test)

In [49]:
train = df.drop('Survived', axis=1)
labels = df.Survived
test = df_test
#データを分割
#(train_X, test_X ,train_y, test_y) = train_test_split(train_X, train_y, test_size = 0.3, random_state = 666)

In [52]:
train.head()

,Sex,Age,SibSp,Parch,Fare,NumFamily,IsFamily,IsPclass1,IsPclass2,IsEmbarkedS,IsEmbarkedC,IsEmbarkedQ
PassengerId,,,,,,,,,,,,
1,1,22.0,1,0,7.2500,1,1.0,0,1,1,0,0
2,0,38.0,1,0,71.2833,1,1.0,1,0,0,1,0
3,0,26.0,0,0,7.9250,0,NaN,0,1,1,0,0
4,0,35.0,1,0,53.1000,1,1.0,1,0,1,0,0
5,1,35.0,0,0,8.0500,0,0.0,0,1,1,0,0


# 以下適当なコード